# Detecting Anomalies In Stationary Combustion Data Through Isolated Forests

## Package Importing

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest

## Database Importing

In [5]:
import dotenv
import os
import psycopg2

dotenv.load_dotenv(dotenv_path='database.env')

# --- Configuration ---
DB_NAME = os.environ['DB_NAME']
DB_USER = os.environ['DB_USER']
DB_PASS = os.environ['DB_PASS']
HOST = os.environ['HOST']
PORT = os.environ['PORT']

# 2. Database Connection and Setup
conn = psycopg2.connect(
    dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=HOST, port=PORT
)
conn.autocommit = True
cur = conn.cursor()

cur.execute('SELECT "facilityId","co2e","fuelHint","createdAt","ch4","n2o","ef","co2" FROM stationary_combustion_activity')

df = pd.DataFrame(cur.fetchall(), columns = ["facilityId","co2e","fuelHint","createdAt","ch4","n2o","ef","co2"])
print(df)

                                facilityId          co2e  \
0     68f4851d-4959-49b6-96a1-63d80c816ed3      0.000000   
1     bf2278d8-0d33-4e28-a55e-ad3c1537af7e      0.000054   
2     4f4bf05a-0a52-48db-b66d-3b3da6b23619      0.000054   
3     4f4bf05a-0a52-48db-b66d-3b3da6b23619      0.004033   
4     4f4bf05a-0a52-48db-b66d-3b3da6b23619      0.010790   
...                                    ...           ...   
6160  beae4852-c882-4706-8dd9-f8ce4dc11989      0.222116   
6161  d0f9256f-17f4-4b7e-abc7-2eeeba8d59c0      0.053958   
6162  d0f9256f-17f4-4b7e-abc7-2eeeba8d59c0   9049.547300   
6163  e0bd2a9c-69d4-403e-beed-3b9a6b08b382  12163.778891   
6164  e0bd2a9c-69d4-403e-beed-3b9a6b08b382   6588.125943   

                           fuelHint                         createdAt  \
0                            Bamboo  2025-01-05 21:53:26.836000-08:00   
1                       Natural Gas  2025-01-20 00:33:24.881000-08:00   
2                       Natural Gas  2025-01-20 02:59:01.888

## Data Cleaning

We will first create hour, day of week, and month-based variables to account for factors like seasonality.

In [7]:
# Convert 'createdAt' to datetime objects
df['createdAt'] = pd.to_datetime(df['createdAt'], utc=True)

# --- 2. Feature Engineering ---

# Extract time-based features
df['hour'] = df['createdAt'].dt.hour
df['day_of_week'] = df['createdAt'].dt.dayofweek # Monday=0, Sunday=6
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
df['month'] = df['createdAt'].dt.month

# For cyclical seasonality, convert month to sine and cosine features
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

This section of code factors in a manual "event log" that can account for changes in production, such as a facility being inactive during a certain period of time, or producing higher rates of a product during a certain busy time, and factoring that into the emission readings accordingly.

In [9]:
# --- 6. Event Log Integration and Feature Creation ---

# 1. Simulate the Event Log (This is where the user/system input goes)
# Example: Facility X increases production by 50% from June 1st to July 31st
event_log = pd.read_csv("sample_event_log.csv")

# 2. Initialize the new feature to the baseline (1.0, meaning no change)
df['production_boost_factor'] = 1.0

# 3. Apply the boost factor for records that fall within event periods
for _, event in event_log.iterrows():
    # Check if a record is from the facility AND within the event's date range
    mask = (df['facilityId'] == event['facilityId']) & \
           (df['createdAt'] >= event['event_start']) & \
           (df['createdAt'] <= event['event_end'])
    
    # Apply the magnitude to the new feature column
    df.loc[mask, 'production_boost_factor'] = event['magnitude']

print(f"Applied production boost to {df[df['production_boost_factor'] > 1.0].shape[0]} records.")

Applied production boost to 69 records.


We keep the necessary features in our data and prepare the data for our model.

In [11]:
# Define the features to be used in the model
NUMERICAL_FEATURES = ['co2e', 'ch4', 'n2o', 'ef', 'co2', 'month_sin', 'month_cos', 'production_boost_factor']
TIME_FEATURES = ['hour', 'day_of_week', 'is_weekend']
CATEGORICAL_FEATURES = ['facilityId', 'fuelHint']

FEATURES = NUMERICAL_FEATURES + TIME_FEATURES + CATEGORICAL_FEATURES
X = df[FEATURES]

# --- 3. Preprocessing Pipeline ---

# Create transformers for different feature types
numerical_transformer = StandardScaler() # Scaling is crucial for Isolation Forest

# One-Hot Encoding for categorical features
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, NUMERICAL_FEATURES + TIME_FEATURES),
        ('cat', categorical_transformer, CATEGORICAL_FEATURES)
    ],
    remainder='drop' # Drop other columns not specified
)


In [22]:
# The Isolation Forest model is typically trained on all data since we don't have labeled anomalies.
# 'contamination' is the expected proportion of anomalies in the dataset (e.g., 1%).
# Setting a reasonable contamination value helps the model set its internal threshold.

# Define the model within a pipeline for clean preprocessing and fitting
anomaly_detector = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', IsolationForest(
        n_estimators=100,
        contamination=0.01, # Set based on domain knowledge (e.g., 1% of data is anomalous)
        random_state=42,
        n_jobs=-1 # Use all available cores
    ))
])

# --- 5. Training and Prediction ---

print("Training Isolation Forest model...")

# Assuming 'df' is your historical data and you want to use the last 6 months
six_months_ago = pd.to_datetime('today', utc=True) - pd.DateOffset(months=6)

# Filter the data for the rolling window
recent_data = df[df['createdAt'] >= six_months_ago]
X_recent = recent_data[FEATURES]

anomaly_detector.fit(X)

# Get the anomaly scores (lower score means more anomalous)
# Note: Isolation Forest outputs a score where higher is 'normal'.
# We often use .decision_function() for the raw score.
df['anomaly_score'] = anomaly_detector.decision_function(X)

# Predict the anomaly classification (-1 for anomaly, 1 for normal)
df['anomaly_label'] = anomaly_detector.predict(X)

print("Training complete.")

Training Isolation Forest model...
Training complete.


## Model Creation

This section is where we define and create our model. We train it to prioritize recent data (such as in the last six months) and fit our data to the model. `X_recent` 

In [18]:
X_recent.head()

,co2e,ch4,n2o,ef,co2,month_sin,month_cos,production_boost_factor,hour,day_of_week,is_weekend,facilityId,fuelHint
2035,0.000000,0.000000,0.000000,95.264500,0.000000,1.224647e-16,-1.0,1.0,15,2,0,26dbdfca-dc1b-45d7-b64a-6d4a73f66b75,Straw
2036,0.000000,0.000000,0.000000,93.864500,0.000000,1.224647e-16,-1.0,1.0,16,2,0,26dbdfca-dc1b-45d7-b64a-6d4a73f66b75,Bamboo
2037,0.011402,0.003036,0.000398,120.179000,0.000000,1.224647e-16,-1.0,1.0,18,2,0,26dbdfca-dc1b-45d7-b64a-6d4a73f66b75,Agricultural Byproducts
2038,0.000000,0.000000,0.000000,95.664500,0.000000,1.224647e-16,-1.0,1.0,19,2,0,26dbdfca-dc1b-45d7-b64a-6d4a73f66b75,Bagasse
2039,0.961533,0.009843,0.002051,46.889940,0.960714,1.224647e-16,-1.0,1.0,20,2,0,26dbdfca-dc1b-45d7-b64a-6d4a73f66b75,Coke Oven Gas


## Results

In [26]:
print("\n--- Results Summary ---")
print(f"Total Anomalies Detected (-1): {df['anomaly_label'].value_counts().get(-1, 0)}")
print(f"Total Normal Observations (1): {df['anomaly_label'].value_counts().get(1, 0)}")

# --- 6. Inspection of Anomalies ---

anomalies = df[df['anomaly_label'] == -1].sort_values(by='anomaly_score')

# Show the top 5 most anomalous observations (lowest scores)
print("\nTop 5 Most Anomalous Records:")
print(anomalies[['createdAt','fuelHint', 'co2e', 'anomaly_score']].head())

# Example of how to filter based on a score threshold if you prefer a continuous approach
# score_threshold = df['anomaly_score'].quantile(0.01) # Set threshold at the 1st percentile
# high_risk = df[df['anomaly_score'] < score_threshold]


--- Results Summary ---
Total Anomalies Detected (-1): 57
Total Normal Observations (1): 6108

Top 5 Most Anomalous Records:
                            createdAt fuelHint            co2e  anomaly_score
1732 2025-03-20 03:56:15.627000+00:00     Coal  3357926.348112      -0.044954
3787 2025-03-20 03:56:15.627000+00:00     Coal  3357926.348112      -0.044954
5842 2025-03-20 03:56:15.627000+00:00     Coal  3357926.348112      -0.044954
2048 2025-07-10 09:22:48.624000+00:00     Coal   170737.098157      -0.025154
4103 2025-07-10 09:22:48.624000+00:00     Coal   170737.098157      -0.025154
